### Inference Test

In [1]:
from __future__ import print_function

import os
import sys

import numpy as np
import tensorflow as tf

sys.path.extend([".", ".."])  # adds the folder from which you call the script
os.environ["CASAPOSE_INFERENCE"] = "True"

from casapose.data_handler.image_only_dataset import ImageOnlyDataset
from casapose.data_handler.vectorfield_dataset import VectorfieldDataset
from casapose.pose_estimation.pose_evaluation import poses_pnp
from casapose.pose_estimation.voting_layers_2d import CoordLSVotingWeighted
from casapose.pose_models.tfkeras import Classifiers
from casapose.utils.config_parser import parse_config

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

/usr/local/lib/python3.8/dist-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)
/usr/local/lib/python3.8/dist-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `FrequencyNoiseAlpha()` is deprecated. Use `BlendAlphaFrequencyNoise` instead. FrequencyNoiseAlpha is deprecated. Use BlendAlphaFrequencyNoise instead. The order of parameters is the same. Parameter 'first' was renamed to 'foreground'. Parameter 'second' was renamed to 'background'.
  warn_deprecated(msg, stacklevel=3)
2023-03-31 13:10:40.260370: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 13:10:40

In [2]:
#opt = parse_config()
#parse_config()

In [3]:
class opt:
    modelname = 'casapose_c_gcu5'
    estimate_confidence = 1
    estimate_coords = 1
    confidence_regularization = 1
    object = 'obj_000001,obj_000005,obj_000006,obj_000008,obj_000009,obj_000010,obj_000011,obj_000016'

    no_points = 9
    save_debug_batch = 0

    imagesize = (448, 448)
    imagesize_test = (480, 640)
    crop_factor = 0.933333333
    pretrained = 1
    manualseed = 1237

    # losses
    mask_loss_weight = 1.0
    vertex_loss_weight = 0.5
    proxy_loss_weight = 0.015
    keypoint_loss_weight = 0.007
    filter_vertex_with_segmentation = 1
    filter_high_proxy_errors = 0
    use_bpnp_reprojection_loss = 0
    max_keypoint_pixel_error = 12.5

    # learning rate
    lr = 0.001
    lr_decay = 0.5
    lr_epochs_steps = 50,75,90

    # general
    gpuids = 0,1
    loginterval = 10
    epochs = 100
    batchsize = 4
    saveinterval = 5
    validationinterval = 1

    # data preprocessing
    workers = 0
    prefetch = 10

    # augmentation
    translation = 0
    rotation = 0
    noise = 0.0001
    brightness = 0.001
    contrast = 0.001
    saturation = 0.001
    hue = 0.001
    use_imgaug = 1

    # test
    min_object_size_test = 200
    write_poses = 0
    save_eval_batches = 0

    # output
    net = 'training_checkpoints'
    outf = 'train_casapose_8_16_objects'

    # config
    train_vectors_with_ground_truth = 1
    load_h5_weights = 0
    copy_weights_from_backup_network = 0
    copy_weights_add_confidence_maps = 0
    objects_in_input_network = 8
    objects_to_copy = 1
    objects_to_copy_list = 'config/objects_to_copy.csv'
    
    confidence_filter_estimates = 1
    confidence_choose_second = 0
    train_vectors_with_ground_truth = 0
    datatest_wxyz_quaterion = 0
    filter_test_with_gt = 0
    
    evalf = 'output'
    #datatest = 'import_data/test/test'
    #datameshes = 'import_data/test/models'
    datatest = '/workspace/CASAPose/import_data/test/test'
    datameshes = '/workspace/CASAPose/import_data/test/models'
    data = ''
    datatest_path_filter = 'None'
    color_dataset = 1
    train_validation_split = 0.9
    backbonename = 'resnet18'
    load_h5_weights = 1
    load_h5_filename = '../../../data/pretrained_models/result_w'

In [4]:
if not os.path.exists(opt.evalf):
    os.makedirs(opt.evalf)

In [5]:
checkpoint_path = opt.outf + "/" + opt.net

frozen_path = opt.outf + "/frozen_model"
img_out_path = opt.outf + "/control_output"

In [6]:
def create_dir(path):
    try:
        os.makedirs(path)
    except OSError:
        pass

In [7]:
create_dir(img_out_path)

In [8]:
# save the hyper parameters passed
with open(opt.evalf + "/header_eval.txt", "w") as file:
    file.write(str(opt))

In [9]:
# set the manual seed.
np.random.seed(opt.manualseed)
tf.random.set_seed(opt.manualseed)

In [10]:
test_dataset = None

device_ids = []
if len(opt.gpuids) == 1 and opt.gpuids[0] < 0:
    device_ids.append("/cpu:0")
else:
    device_ids.append("/gpu:{}".format(opt.gpuids[0]))
print(device_ids)

['/gpu:0']


In [11]:
objectsofinterest = [x.strip() for x in opt.object.split(",")]
no_objects = len(objectsofinterest)
separated_vectorfields = opt.modelname == "pvnet"

In [12]:
objectsofinterest

['obj_000001',
 'obj_000005',
 'obj_000006',
 'obj_000008',
 'obj_000009',
 'obj_000010',
 'obj_000011',
 'obj_000016']

In [13]:
testingdata = None
normal_imgs = [0.5, 0.5]

use_split = False
if opt.data == opt.datatest:
    print("split datasets with ratio {}".format(opt.train_validation_split))
    use_split = True

test_batches = 0

In [14]:
test_dataset = ImageOnlyDataset(root=opt.datatest)
testing_images, _ = test_dataset.generate_dataset(1)

2023-03-31 13:11:13.759056: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 13:11:13.759976: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 13:11:13.760712: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 13:11:13.761658: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 13:11:13.762380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from S

In [15]:
ImageOnlyDataset(root=opt.datatest).imgs

['/workspace/CASAPose/import_data/test/test/000001/000001.jpg',
 '/workspace/CASAPose/import_data/test/test/000001/000002.jpg',
 '/workspace/CASAPose/import_data/test/test/000001/000003.jpg',
 '/workspace/CASAPose/import_data/test/test/000001/000004.jpg',
 '/workspace/CASAPose/import_data/test/test/000001/000005.jpg']

In [16]:
len(ImageOnlyDataset(root=opt.datatest))

5

In [17]:
len(test_dataset)

5

In [18]:
len(testing_images)

5

In [19]:
test_dataset = VectorfieldDataset(
    root=opt.datatest,
    path_meshes=opt.datameshes,
    path_filter_root=opt.datatest_path_filter,
    color_input=opt.color_dataset,
    no_points=opt.no_points,
    objectsofinterest=objectsofinterest,
    noise=0.00001,
    data_size=None,
    save=opt.save_debug_batch,
    normal=normal_imgs,
    contrast=0.00001,
    brightness=0.00001,
    hue=0.00001,
    saturation=0.00001,
    random_translation=(0, 0),
    random_rotation=0,
    random_crop=False,
    use_validation_split=use_split,
    use_train_split=False,
    train_validation_split=opt.train_validation_split,
    output_folder=opt.evalf,
    visibility_filter=False,
    separated_vectorfields=(opt.modelname == "pvnet"),
)

/workspace/CASAPose/import_data/test/models/obj_000001/obj_000001.ply
/workspace/CASAPose/import_data/test/models/obj_000002/obj_000002.ply
/workspace/CASAPose/import_data/test/models/obj_000003/obj_000003.ply
/workspace/CASAPose/import_data/test/models/obj_000004/obj_000004.ply
/workspace/CASAPose/import_data/test/models/obj_000005/obj_000005.ply
/workspace/CASAPose/import_data/test/models/obj_000006/obj_000006.ply
/workspace/CASAPose/import_data/test/models/obj_000007/obj_000007.ply
/workspace/CASAPose/import_data/test/models/obj_000008/obj_000008.ply
/workspace/CASAPose/import_data/test/models/obj_000009/obj_000009.ply
/workspace/CASAPose/import_data/test/models/obj_000010/obj_000010.ply
/workspace/CASAPose/import_data/test/models/obj_000011/obj_000011.ply
/workspace/CASAPose/import_data/test/models/obj_000012/obj_000012.ply
/workspace/CASAPose/import_data/test/models/obj_000013/obj_000013.ply
/workspace/CASAPose/import_data/test/models/obj_000014/obj_000014.ply
/workspace/CASAPose/

In [20]:
test_dataset = VectorfieldDataset(
    root="/workspace/CASAPose/import_data/test/test", # Hardcoded param
    path_meshes="/workspace/CASAPose/import_data/test/models", # Hardcoded param
    path_filter_root=None, # Hardcoded param
    color_input=True, # Hardcoded param
    no_points=9, # Taken from config_8.ini
    objectsofinterest=objectsofinterest,
    noise=0.00001,
    data_size=None,
    save=False, # Hardcoded param
    normal=normal_imgs,
    contrast=0.00001,
    brightness=0.00001,
    hue=0.00001,
    saturation=0.00001,
    random_translation=(0, 0),
    random_rotation=0,
    random_crop=False,
    use_validation_split=use_split,
    use_train_split=False,
    train_validation_split=None,
    output_folder=None,
    visibility_filter=False,
    separated_vectorfields=(opt.modelname == "pvnet"),
)

/workspace/CASAPose/import_data/test/models/obj_000001/obj_000001.ply
/workspace/CASAPose/import_data/test/models/obj_000002/obj_000002.ply
/workspace/CASAPose/import_data/test/models/obj_000003/obj_000003.ply
/workspace/CASAPose/import_data/test/models/obj_000004/obj_000004.ply
/workspace/CASAPose/import_data/test/models/obj_000005/obj_000005.ply
/workspace/CASAPose/import_data/test/models/obj_000006/obj_000006.ply
/workspace/CASAPose/import_data/test/models/obj_000007/obj_000007.ply
/workspace/CASAPose/import_data/test/models/obj_000008/obj_000008.ply
/workspace/CASAPose/import_data/test/models/obj_000009/obj_000009.ply
/workspace/CASAPose/import_data/test/models/obj_000010/obj_000010.ply
/workspace/CASAPose/import_data/test/models/obj_000011/obj_000011.ply
/workspace/CASAPose/import_data/test/models/obj_000012/obj_000012.ply
/workspace/CASAPose/import_data/test/models/obj_000013/obj_000013.ply
/workspace/CASAPose/import_data/test/models/obj_000014/obj_000014.ply
/workspace/CASAPose/

In [21]:
no_objects

8

In [22]:
len(test_dataset)

5

In [23]:
print(len(test_dataset))
print(len(testing_images))
testingdata, test_batches = test_dataset.generate_dataset(
    1, 1, 0, opt.imagesize_test, 1.0, 1, no_objects, shuffle=False
)

#'epochs', 'prefetch', 'imagesize', 'cropratio', 'worker', and 'no_objects'

5
5
Instructions for updating:
Use fn_output_signature instead


In [24]:
mesh_vertex_array, mesh_vertex_count = test_dataset.generate_object_vertex_array()

print("testing data: {} batches".format(test_batches))

testing data: 5.0 batches


In [25]:
input_segmentation_shape = None
if opt.train_vectors_with_ground_truth is True:
    input_segmentation_shape = (
        opt.imagesize_test[0],
        opt.imagesize_test[1],
        1 + no_objects,
    )

In [26]:
height = opt.imagesize_test[0]
width = opt.imagesize_test[1]

In [27]:
width

640

In [28]:
CASAPose = Classifiers.get(opt.modelname)

In [29]:
ver_dim = opt.no_points * 2

In [30]:
ver_dim

18

In [31]:
if opt.modelname == "pvnet":
    ver_dim = ver_dim * no_objects

if opt.estimate_confidence:
    assert separated_vectorfields is not None, "confidence not compaitble with this model"
    ver_dim += opt.no_points

In [32]:
net = CASAPose(
    ver_dim=ver_dim,
    seg_dim=1 + no_objects,
    input_shape=(height, width, 3),
    input_segmentation_shape=input_segmentation_shape,
    weights="imagenet",
    base_model=opt.backbonename,
)

[<KerasTensor: shape=(None, 240, 320, 64) dtype=float32 (created by layer 'relu0')>, <KerasTensor: shape=(None, 120, 160, 64) dtype=float32 (created by layer 'stage2_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 128) dtype=float32 (created by layer 'stage3_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 256) dtype=float32 (created by layer 'stage4_unit1_relu1')>, <KerasTensor: shape=(None, 60, 80, 512) dtype=float32 (created by layer 'relu1')>]


In [33]:
checkpoint_prefix = os.path.join(checkpoint_path, "ckpt")
checkpoint = tf.train.Checkpoint(network=net)  # , optimizer=optimizer)

In [34]:
if opt.load_h5_weights is True:
    net.load_weights(frozen_path + "/" + opt.load_h5_filename + ".h5", by_name=True, skip_mismatch=True)

elif opt.net != "":
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path)).expect_partial()

for layer in net.layers:
    layer.trainable = False

In [35]:
net.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 480, 640, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             [(None, 240, 320, 6  11186889    ['data[0][0]']                   
                                4),                                                               
                                 (None, 120, 160, 6                                               
                                4),                                                               
                                 (None, 60, 80, 128                                         

In [36]:
with open(opt.evalf + "/speed_eval.csv", "w") as file:
    file.write("batchid,speed \n")

In [37]:
def runnetwork(loader_iterator, batches, keypoints, camera_matrix, no_objects):
    @tf.function
    def test_step(img_batch):
        start_time = tf.timestamp()
        no_points = opt.no_points
        net_input = [tf.expand_dims(img_batch[0], 0)]
        output_net = net(net_input, training=False)  # all stages are present here
        output_seg, output_dirs, confidence = tf.split(output_net, [no_objects, no_points * 2, -1], 3)
        coordLSV_in = [output_seg, output_dirs, confidence]
        coords = CoordLSVotingWeighted(
            name="coords_ls_voting",
            num_classes=no_objects,
            num_points=no_points,
            filter_estimates=True,
        )(coordLSV_in)
        poses_est = poses_pnp(
            coords, output_seg, keypoints, camera_matrix, no_objects - 1, min_num=opt.min_object_size_test
        )
        tf.print(poses_est[0, 0, 0, 0, 0])  # access memory to get sure everything is done

        end_time = tf.timestamp()
        time_needed = end_time - start_time
        return time_needed

    speed = []
    for batch_idx in range(batches):
        img_batch = loader_iterator.get_next()
        time_needed = test_step(img_batch)
        speed.append(time_needed)
        with open(opt.evalf + "/speed_eval.csv", "a") as file:
            s = "{},{:.7f}\n".format(batch_idx + 1, time_needed)
            file.write(s)

    tf.print("average speed: {}".format(tf.reduce_mean(speed[10:])), summarize=-1)

    return loader_iterator

In [38]:
print("Test Batches: {} ".format(test_batches))

Test Batches: 5.0 


In [39]:
testingdata_iterator = iter(testingdata)

In [40]:
img_batch = testingdata_iterator.get_next()

In [41]:
keypoints = img_batch[4]
camera_matrix = img_batch[5]

In [42]:
testingdata_iterator = iter(testing_images)
runnetwork(testingdata_iterator, int(test_batches), keypoints, camera_matrix, no_objects + 1)

[TensorShape([8, 9, 2]), TensorShape([8, 9, 3]), TensorShape([3, 3])]


2023-03-31 13:13:20.044503: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-03-31 13:13:21.063519: I tensorflow/stream_executor/cuda/cuda_blas.cc:1804] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-31 13:13:21.288811: I tensorflow/compiler/xla/service/service.cc:170] XLA service 0x7f19835f47a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-31 13:13:21.288860: I tensorflow/compiler/xla/service/service.cc:178]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2023-03-31 13:13:21.294565: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:263] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-31 13:13:21.836268: I tensorflow/compiler/jit/xla_compilation_cache.cc:481] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-03-31 13:13:25.6763

-0.925360084
0.716837
0.693645954
-0.524748921
-0.0622993708
average speed: nan
